In [1]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [3]:
rules = """Правила формирования системы управления рисками и внутреннего контроля для банков второго уровня
Глава 1. Общие положения

      1. Настоящие Правила формирования системы управления рисками и внутреннего контроля для банков второго уровня (далее – Правила), разработаны в соответствии с Законом Республики Казахстан от 31 августа 1995 года "О банках и банковской деятельности в Республике Казахстан" (далее – Закон о банках) и устанавливают порядок формирования системы управления рисками и внутреннего контроля банков второго уровня (далее – банк).

      2. В Правилах используются следующие понятия:

      1) риск информационной безопасности – вероятное возникновение ущерба вследствие нарушения конфиденциальности, преднамеренного нарушения целостности или доступности информационных активов банка;

      2) риск информационных технологий – вероятность возникновения ущерба вследствие отказа (нарушения функционирования) информационно-коммуникационных технологий, эксплуатируемых банком;

      3) уполномоченный коллегиальный орган банка – совет директоров, комитет при совете директоров, правление, комитет при правлении;

      4) репутационный риск – вероятность возникновения потерь, неполучения запланированных доходов в результате сужения клиентской базы, снижения иных показателей развития вследствие формирования в обществе отрицательного представления о надежности банка, качестве оказываемых им услуг или характере деятельности банка в целом;

      5) юридический риск – вероятность возникновения потерь вследствие несоблюдения банком либо контрагентом требований гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах, а в отношениях с нерезидентами Республики Казахстан - законодательства страны его происхождения, а также условий заключенных договоров;

      6) внутренний процесс оценки достаточности капитала – набор процессов управления существенными рисками, с учетом объема активов, характера и уровня сложности деятельности, организационной структуры, стратегических планов, риск-профиля банка, нормативной правовой базы, оценка и агрегирование таких рисков с целью определения целевого уровня достаточности капитала банка для поддержания стабильного финансового положения и платежеспособности.

      Под капиталом филиала банка-нерезидента Республики Казахстан понимаются активы филиала банка-нерезидента Республики Казахстан, принимаемые в качестве резерва, рассчитанные в соответствии с требованиями постановления Правления Агентства Республики Казахстан по регулированию и развитию финансового рынка от 12 февраля 2021 года № 23 "Об установлении пруденциальных нормативов и иных обязательных к соблюдению норм и лимитов для филиалов банков-нерезидентов Республики Казахстан (в том числе филиалов исламских банков-нерезидентов Республики Казахстан), их нормативных значений и методики расчетов, включая порядок формирования активов филиалов банков-нерезидентов Республики Казахстан (в том числе филиалов исламских банков-нерезидентов Республики Казахстан), принимаемых в качестве резерва, и их минимальный размер", зарегистрированного в Реестре государственной регистрации нормативных правовых актов под № 22213;

      7) план финансирования капитала – совокупность процедур и плана действий для реагирования на критическое снижение капитала банка;

      8) статистический журнал стоимости залогового обеспечения – внутренний журнал стоимостей залогового обеспечения, включающий описание и характеристику залогового обеспечения, информацию по итогам первой и наиболее актуальной оценок независимой оценки качества (дата оценки, наименование независимой оценки качества, стоимость, метод оценки), заключения залоговой службы (дата, стоимость), причины различия стоимостей, информацию по реализации (при наличии);

      9) беззалоговый потребительский заем – банковский заем без условия о залоге на момент выдачи, предоставленный физическому лицу на цели, не связанные с осуществлением предпринимательской деятельности;

      10) комплаенс-риск – вероятность возникновения потерь вследствие несоблюдения банком и его работниками требований гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах, внутренних документов банка, регламентирующих порядок оказания банком услуг и проведения операций на финансовом рынке, а также законодательства иностранных государств, оказывающего влияние на деятельность банка;

      11) корпоративное управление – система взаимоотношений между правлением банка (соответствующим исполнительным органом банка-нерезидента Республики Казахстан, филиал которого открыт на территории Республики Казахстан, руководящими работниками филиала банка-нерезидента Республики Казахстан), советом директоров (соответствующим органом управления банка-нерезидента Республики Казахстан, филиал которого открыт на территории Республики Казахстан) акционерами, руководящими работниками и аудиторами, а также взаимоотношения между уполномоченными коллегиальными органами банка.

      Система корпоративного управления позволяет организовать распределение полномочий и ответственности, а также построить процесс принятия корпоративных решений;

      12) кредитный риск – вероятность возникновения потерь, возникающая вследствие невыполнения заемщиком или контрагентом своих обязательств в соответствии с условиями договора банковского займа;

      13) кредитоспособность – комплексная правовая и финансовая характеристика заемщика, представленная финансовыми и нефинансовыми показателями, позволяющая оценить его возможность в будущем полностью и в срок исполнить обязательства по договору банковского займа;

      14) кредитный договор – соглашение между банком и заемщиком о предоставлении финансирования (включая условное финансирование), в результате которого у банка возникают (либо возникнут в будущем) требования к заемщику;

      15) план финансирования на случай непредвиденных обстоятельств – совокупность процедур и плана действий для реагирования на снижение способности банка своевременно отвечать по своим обязательствам;

      16) надзорное стресс-тестирование – это инструмент уполномоченного органа, направленный на оценку финансовой устойчивости банков к гипотетическим (стрессовым) сценариям развития событий. Банки на основании единой для всех участников надзорного стресс-тестирования методологии и сценариев проводят расчеты с использованием внутренних моделей и предоставляют в уполномоченный орган результаты стресс-тестирования. При этом банки отвечают за надлежащее качество проведенных расчетов и результаты стресс-тестирования;

      17) уполномоченный орган по финансовому мониторингу – государственный орган, осуществляющий финансовый мониторинг и принимающий иные меры по противодействию легализации (отмыванию) доходов, полученных преступным путем, финансированию терроризма, финансированию распространения оружия массового уничтожения;

      18) подразделение-владелец защищаемой информации – подразделение банка, владелец информации, нарушение конфиденциальности, целостности или доступности которой приведет к убыткам для банка;

      19) критичный информационный актив – информационный актив, определяемый в соответствии с постановлением Правления Национального Банка Республики Казахстан от 27 марта 2018 года № 48 "Об утверждении Требований к обеспечению информационной безопасности банков, филиалов банков-нерезидентов Республики Казахстан и организаций, осуществляющих отдельные виды банковских операций, Правил и сроков предоставления информации об инцидентах информационной безопасности, включая сведения о нарушениях, сбоях в информационных системах", зарегистрированным в Реестре государственной регистрации нормативных правовых актов под № 16772;

      20) существенный риск – риск, реализация которого приведет к ухудшению финансовой устойчивости банка;

      21) конфликт интересов – ситуация, при которой возникает противоречие между личной заинтересованностью должностных лиц банка (должностных лиц органа управления, исполнительного органа банка-нерезидента Республики Казахстан, филиал которого открыт на территории Республики Казахстан, руководящих работников филиала банка-нерезидента Республики Казахстан), его акционеров и (или) его работников и надлежащим исполнением ими своих должностных полномочий или имущественными и иными интересами банка и (или) его работников и (или) клиентов, которое повлечет за собой неблагоприятные последствия для банка и (или) его клиентов;

      22) рыночный риск – вероятность возникновения финансовых потерь по балансовым и внебалансовым статьям, обусловленная неблагоприятными изменениями рыночной ситуации, выражающаяся в изменениях рыночных процентных ставок, курсов иностранных валют, рыночной стоимости финансовых инструментов, товаров;

      23) операционный риск – вероятность возникновения потерь в результате неадекватных и недостаточных внутренних процессов, человеческих ресурсов и систем, или влияния внешних событий, за исключением стратегического риска и репутационного риска;

      24) внутренний процесс оценки достаточности ликвидности – набор процессов управления риском ликвидности, в целях поддержания банком надлежащего уровня ликвидности и внедрения надлежащей системы управления риском ликвидности в различные временные интервалы в зависимости от видов деятельности, валюты;

      25) риск ликвидности – вероятность возникновения финансовых потерь в результате неспособности банка выполнить свои обязательства в установленный срок без значительных убытков;

      26) процентный риск – риск возникновения финансовых расходов (убытков) вследствие неблагоприятного изменения процентных ставок по активам, пассивам банка;

      27) политика – утвержденный советом директоров банка внутренний документ, определяющий основные количественные и качественные параметры, принципы, стандарты, обеспечивающие эффективное функционирование банка и соответствие его деятельности стратегии, риск-профилю, риск-аппетиту. В рамках политики совет директоров банка обеспечивает наличие соответствующих внутренних документов, описывающие отдельные процедуры, процессы, инструкции;

      28) стратегический риск – вероятность возникновения убытков в результате ошибок (недостатков), допущенных при принятии решений, определяющих стратегическое развитие банка и выражающихся в недостаточном учете возможных опасностей, присущих деятельности банка, неправильном или недостаточно обоснованном определении перспективных направлений деятельности, в которых банк достигнет преимущества перед конкурентами, отсутствии или обеспечении в неполном объеме необходимых ресурсов и организационных мер, обеспечивающих достижение стратегических целей деятельности банка;

      29) стресс-тестирование – метод оценки потенциального влияния исключительных, но возможных событий на финансовое состояние банка;

      30) риск – вероятность того, что ожидаемые или непредвиденные события окажут отрицательное влияние на финансовую устойчивость банка, его капитал и (или) доходы;

      31) риск-профиль – совокупность видов риска и иных сведений, характеризующих степень подверженности банка рискам, присущим всем видам деятельности банка для выявления слабых сторон и определения приоритетности последующих действий в рамках системы управления рисками;

      32) риск-аппетит – агрегированный (агрегированные) уровень (уровни) существенных рисков (лимиты допустимого размера риска), который (которые) банк готов принять либо намерен исключить при реализации стратегии;

      33) заявление риск-аппетита – утверждаемый советом директоров банка документ, описывающий агрегированный (агрегированные) уровень (уровни) существенных рисков (лимиты допустимого размера риска), который (которые) банк готов принять либо намерен исключить при реализации стратегии. Заявление риск-аппетита содержит заявление качественного характера, а также количественного характера, включая показатели в отношении доходности, капитала, ликвидности, рисков, иных применимых показателей;

      34) риск-культура – процессы, процедуры, внутренние правила банка, направленные на понимание, принятие, управление и контроль за рисками с целью минимизации их влияния на финансовое состояние банка, а также этические нормы и стандарты профессиональной деятельности всех участников организационной структуры. Риск-культура дополняет существующие утвержденные процедуры, процессы и механизмы деятельности банка и является неотъемлемым компонентом системы управления рисками;

      35) обработка риска – процесс выбора и реализации мер по изменению рисков;

      36) реестр рисков – структурированный перечень рисков, содержащий критерии и причины возникновения рисков, вероятность их возникновения, воздействие (ущерб), приоритет и способы обработки риска;

      37) уполномоченный орган – государственный орган, осуществляющий государственное регулирование, контроль и надзор финансового рынка и финансовых организаций;

      38) организационная структура – внутренний документ и (или) совокупность внутренних документов, устанавливающих количественный состав и систему органов управления, руководящих работников и структурных подразделений банка, отражающий структуру подчиненности, подотчетности;

      39) участник Международного финансового центра "Астана", оказывающий услуги по управлению платформой цифровых активов– юридическое лицо, зарегистрированное в соответствии с действующим правом Международного финансового центра "Астана" и осуществляющее на территории Международного финансового центра "Астана" деятельность по управлению платформой цифровых активов;

      40) внутренний (экономический) капитал – капитал, необходимый для покрытия существенных рисков, в том числе потенциальных, принятых банком, рассчитанный внутри банка с использованием собственных моделей.

      При применении требований Правил к филиалу банка-нерезидента Республики Казахстан:

      под советом директоров понимается соответствующий орган управления банка-нерезидента Республики Казахстан;

      под правлением понимаются руководящие работники филиала банка-нерезидента Республики Казахстан;

      под собственным капиталом понимаются активы филиала банка-нерезидента Республики Казахстан, принимаемые в качестве резерва, рассчитанные в соответствии с требованиями постановления Правления Агентства Республики Казахстан по регулированию и развитию финансового рынка от 12 февраля 2021 года № 23 "Об установлении пруденциальных нормативов и иных обязательных к соблюдению норм и лимитов для филиалов банков-нерезидентов Республики Казахстан (в том числе филиалов исламских банков-нерезидентов Республики Казахстан), их нормативных значений и методики расчетов, включая порядок формирования активов филиалов банков-нерезидентов Республики Казахстан (в том числе филиалов исламских банков-нерезидентов Республики Казахстан), принимаемых в качестве резерва, и их минимальный размер", зарегистрированного в Реестре государственной регистрации нормативных правовых актов под № 22213;

      под финансовой отчетностью понимается отчетность по данным бухгалтерского учета филиала банка-нерезидента Республики Казахстан;

      под главой риск-менеджмента понимается руководитель подразделения по управлению рисками филиала банка-нерезидента Республики Казахстан;

      под главным комплаенс-контроллером понимается руководитель подразделения по комплаенс-контролю филиала банка-нерезидента Республики Казахстан.
      Сноска. Пункт 2 - в редакции постановления Правления Агентства РК по регулированию и развитию финансового рынка от 29.12.2022 № 119 (вводится в действие по истечении десяти календарных дней после дня его первого официального опубликования).

      3. Целью Правил является определение требований к формированию банком систем управления рисками и внутреннего контроля путем обеспечения:

      1) эффективного управления рисками банка посредством своевременного их выявления, измерения, контроля и мониторинга для обеспечения соответствия собственного капитала банка уровню принимаемых им рисков и наличия соответствующего уровня ликвидности;

      2) надлежащей практики корпоративного управления и надлежащего уровня деловой этики и риск-культуры;

      3) соблюдения банком и его работниками требований гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах, внутренних политик, процедур и иных внутренних документов банка;

      4) своевременного обнаружения и устранения недостатков в деятельности банка и его работников;

      5) создания в банке адекватных механизмов для решения непредвиденных или чрезвычайных ситуаций.

      4. Совет директоров банка обеспечивает наличие системы управления рисками, соответствующей выбранной бизнес модели, масштабу деятельности, видам и сложности операций и обеспечивает надлежащий процесс выявления, измерения и оценки, мониторинга, контроля и процедур минимизации существенных рисков банка с целью определения банком величины собственного капитала и ликвидности, необходимых для покрытия существенных рисков, присущих деятельности банка.

      Система управления рисками представляет собой совокупность компонентов, установленных Правилами, которая обеспечивает механизм взаимодействия разработанных и регламентированных банком внутренних процедур, процессов, политик, структурных подразделений банка с целью своевременного выявления, измерения, контроля и мониторинга рисков банка, а также их минимизации для обеспечения его финансовой устойчивости и стабильного функционирования.

      5. Система управления рисками обеспечивает:

      1) оптимальное соотношение между доходностью основных направлений деятельности банка и уровнем принимаемых рисков, основанное на выборе жизнеспособной и устойчивой бизнес модели, эффективном процессе планирования стратегии и бюджета с учетом стратегии риск-аппетита;

      2) объективную оценку размера рисков банка, полноту и документирование процессов управления рисками, их превентивного выявления, измерения и оценки, мониторинга и контроля, минимизацию существенных видов рисков на каждом уровне организационной структуры с оптимальным использованием финансовых ресурсов, персонала и информационных систем в целях поддержания достаточного объема собственного капитала банка и ликвидности;

      3) охват всех видов деятельности банка, подверженных существенным рискам, на всех уровнях организационной структуры, полноту оценки отдельных существенных видов рисков, их взаимного влияния в целях определения риск-профиля банка и построения стратегии риск-аппетита;

      4) наличие уровней риск-аппетита по всем видам существенных рисков и алгоритм действий в случаях нарушения установленных уровней, включая ответственность за принятие рисков, уровень которых определен как высокий, процедуры по информированию совета директоров, комитетов при совете директоров и правления банка в рамках стратегии риск-аппетита;

      5) осведомленность уполномоченных коллегиальных органов банка, принимающих решения, несущие за собой риски, посредством построения эффективной системы корпоративного управления, наличия полной, достоверной и своевременной управленческой информации о существенных рисках, присущих деятельности банка;

      6) рациональное принятие решений и действие в интересах банка на основании всесторонней оценки предоставляемой информации добросовестно, с должной осмотрительностью и заботливостью (duty of care). Обязанность проявлять осмотрительность и заботливость не распространяется на ошибки в процессе принятия бизнес-решений, если только работники и должностные лица банка не проявили при этом грубую небрежность;

      7) принятие решений работниками и должностными лицами банка и действие добросовестно в интересах банка, не учитывая личные выгоды, интересы лиц, связанных с банком особыми отношениями, в ущерб интересов банка (duty of loyalty);

      8) четкое распределение функций, обязанностей и полномочий управления рисками между всеми структурными подразделениями и работниками банка, и их ответственности с учетом минимизации конфликта интересов;

      9) разделение функции управления рисками и внутреннего контроля от операционной деятельности банка посредством построения системы трех линий защиты, которая включает:

      первую линию - на уровне структурных подразделений банка;

      вторую линию - на уровне подразделений по управлению рисками и выполняющих контрольные функции;

      третью линию - на уровне подразделения внутреннего аудита в части оценки эффективности функционирования системы управления рисками;

      10) наличие документов, разработанных в целях регламентирования деятельности банка, создания и функционирования в банке эффективных систем управления рисками и внутреннего контроля и соответствующих стратегии, организационной структуре, профилю рисков банка и требованиям гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах, а также их периодический пересмотр и актуализацию;

      11) соблюдение требований гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах;

      12) соблюдение действующих процедур, процессов, политик и иных внутренних документов банка по управлению рисками посредством построения эффективной системы внутреннего контроля.

      6. Уполномоченный орган в рамках оценки эффективности системы управления рисками банка руководствуется следующими принципами:

      1) обеспечение финансовой стабильности банков, недопущения ухудшения финансового положения банков и увеличения рисков, связанных с деятельностью банков, защита законных интересов депозиторов, кредиторов, клиентов и корреспондентов банков;

      2) преобладание сущности над формой, выражающееся в оценке системы управления рисками банка как механизма измерения и оценки, мониторинга, контроля, и минимизации существенных рисков банка, а не формально регламентированных процедур банка и соблюдения требований гражданского, налогового, банковского законодательства Республики Казахстан, законодательства Республики Казахстан о государственном регулировании, контроле и надзоре финансового рынка и финансовых организаций, законодательства Республики Казахстан о валютном регулировании и валютном контроле, о платежах и платежных системах, о пенсионном обеспечении, о рынке ценных бумаг, о бухгалтерском учете и финансовой отчетности, о кредитных бюро и формировании кредитных историй, о коллекторской деятельности, об обязательном гарантировании депозитов, о противодействии легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма, об акционерных обществах, внутренних документов банка;

      3) пропорциональность при осуществлении функций по контролю и надзору, а также при применении по результатам контроля и надзора, мер предусмотренных законами Республики Казахстан, исходя из принятой банком бизнес модели, масштаба деятельности, видов и сложности операций и существенности рисков банка;

      4) применение единообразного подхода к оценке системы управления рисками и мерам надзорного реагирования;

      5) выявление существенных рисков в деятельности банка.

      7. Уполномоченный орган осуществляет оценку:

      1) эффективности системы корпоративного управления;

      2) существенных рисков, присущих деятельности банка, с учетом видов и сложности операций банка;

      3) соответствия систем управления рисками выбранной бизнес модели, масштабу деятельности, видам и сложности операций банка;

      4) финансового состояния крупных участников банка в целях определения возможности поддержания финансовой устойчивости банка;

      5) влияния финансового состояния участников банковского конгломерата на финансовую устойчивость банка;

      6) эффективности применения превентивных мер с целью недопущения ухудшения финансовой устойчивости банка посредством корректировки систем управления рисками, исходя из масштаба деятельности и уровня принимаемых рисков;

      7) применения системы количественных и качественных показателей в рамках оценки деятельности банка и эффективность методов моделирования.
Глава 2. Бизнес модель

      8. Бизнес модель банка – это совокупность выбранной стратегии, продуктов, процессов планирования, обеспечивающих конкурентоспособность и достаточный уровень доходности. Основными принципами при формировании бизнес модели банка являются:

      1) жизнеспособность, выражающаяся в способности банка обеспечивать достаточный уровень доходности в ближайшие 12 (двенадцать) месяцев и основанная на бюджетном планировании и прогнозировании финансовых показателей;

      2) устойчивость, выражающаяся в способности банка обеспечивать достаточный уровень доходности на период не менее 3 (трех) лет и основанная на стратегическом планировании и прогнозировании финансовых показателей.

      Банк проводит регулярный анализ бизнес модели в целях оценки влияния на нее стратегических рисков и рисков, присущих деятельности банка.

      Банковская деятельность осуществляется в рамках выбранной бизнес модели с учетом объема активов, характера и уровня сложности деятельности, организационной структуры, риск-профиля.

      9. Стратегия банка утверждается советом директоров банка на период не менее 3 (трех) лет и содержит:

      1) миссии и цели развития деятельности банка. Цели являются измеримыми, достижимыми, реалистичными и имеющими точные сроки реализации;

      2) целевые сегменты рынка в разрезе секторов экономики и географического распределения развития деятельности банка;

      3) анализ сильных и слабых сторон выбранной стратегии банка с учетом ключевых источников доходов;

      4) количественные показатели кредитного портфеля, ликвидных активов, вкладов клиентов и других привлеченных средств с учетом установленных уровней риск-аппетита. При этом используемые реалистичные допущения, которые учитывают имеющиеся и доступные ресурсы, текущие и потенциальные экономические условия;

      5) анализ ключевых источников доходов;

      6) ключевые виды вложений, их структуру и планируемые изменения, в том числе по внедрению и развитию новых продуктов и услуг с учетом оценки рисков и процессов, связанных с их внедрением и развитием, а также оценки текущих возможностей банка по внедрению и развитию таких продуктов;

      7) сценарии стратегического развития деятельности банка (негативный, и наиболее вероятные варианты развития событий).

      10. Бюджет банка ежегодно утверждается советом директоров банка и содержит ежемесячный прогноз финансовых показателей (активов и пассивов, доходов и расходов, информацию о ссудном портфеле, вкладах клиентов и иных привлеченных средствах, в разрезе валют (национальной и иностранных валют в совокупности), категорий клиентов).

      Бюджет соответствует стратегии банка. При этом используемые допущения реалистичны и учитывают имеющиеся и доступные ресурсы, текущие и потенциальные экономические условия и возможные риски.

      Одним из компонентов эффективного бюджетного планирования является тарифная политика, которая минимально включает следующие компоненты:

      внутренний порядок и процедуры проведения рыночного анализа спроса и цен на банковские услуги;

      внутренний порядок и процедуры формирования структуры процентных ставок и тарифов;

      приемлемые для банка предельные нижние и верхние границы процентных ставок и тарифов, а также требования к внутреннему порядку их утверждения с учетом требований гражданского, банковского законодательства Республики Казахстан, о платежах и платежных системах, об обязательном гарантировании депозитов, их применения и периодического пересмотра;

      критерии выбора метода определения цен на банковские услуги, а также требования к методикам, основанным на оценке характера и уровня сложности деятельности банка и рисков, присущих банку;

      участники процесса ценообразования и порядок взаимодействия между ними, включая обмен информацией;

      внутренний порядок и процедуры своевременного информирования клиентов банка об условиях предоставления банковских услуг, а также информирование об изменениях.

      Банк ежемесячно проводит анализ бюджета на предмет соответствия прогнозных показателей фактическим значениям, причин выявленных отклонений с последующей разработкой при необходимости корректирующих мер по исправлению и вносит обоснованные корректировки с дальнейшим их документированием.

      11. В процессе стратегического и бюджетного планирования банк проводит анализ ключевых источников доходности с целью выявления потенциальных рисков.

      В целях поддержания стратегии и бюджета банка в актуальном состоянии банк ежегодно осуществляет анализ целевых рынков, в которых осуществляет свою деятельность, проводит оценку конкурентной среды, достаточности ресурсов и способности генерировать краткосрочную и долгосрочную доходность.

      Стратегическое и бюджетное планирование осуществляется в рамках принятых и утвержденных уровней риск-аппетита.
"""

In [4]:
query = f"""Используйте приведенную ниже статью о Правилах формирования системы управления рисками и внутреннего контроля для банков второго уровня для ответа на последующий вопрос. Если ответ не может быть найден, напишите "Я не знаю".

Статья:
\"\"\"
{rules}
\"\"\"

Вопрос: Какие есть основные правила формирования системы управления рисками и внутреннего контроля для банков второго уровня?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'Вы отвечаете на вопрос про правила формирования системы управления рисками и внутреннего контроля для банков второго уровня.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

There were three events in curling at the 2022 Winter Olympics, so there were three sets of athletes who won gold medals. The gold medalists in men's curling were Sweden's Niklas Edin, Oskar Eriksson, Rasmus Wranå, Christoffer Sundgren, and Daniel Magnusson. The gold medalists in women's curling were Great Britain's Eve Muirhead, Vicky Wright, Jennifer Dodds, Hailey Duff, and Mili Smith. The gold medalists in mixed doubles curling were Italy's Stefania Constantini and Amos Mosaner.
